In [16]:
# Importar librerias necesarias
import requests
import sys
from collections import defaultdict, Counter
from datetime import datetime
import json
from dotenv import load_dotenv
import os

load_dotenv() 
#lumu-client-key y collector-id
LUMU_CLIENT_KEY = os.getenv('LUMU_CLIENT_KEY')
COLLECTOR_ID = os.getenv('COLLECTOR_ID')

#Para construir la URL se siguio la estructura que se plantea en la docuemntacion de la API
API_URL = f'https://api.lumu.io/collectors/{COLLECTOR_ID}/dns/queries?key={LUMU_CLIENT_KEY}'


# Funcion que realiza peticion POST a la API
def send_data_api(payload):
    headers = {
        'Content-Type': 'application/json',
        'lumu-client-key': LUMU_CLIENT_KEY
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code != 200:
        print(f"Error al enviar los datos")
    return response


#Ruta del archivo a procesar, como se encuentra en el mismo directorio del .py solo se especifica el nombre del archivo
file_path = 'queries.txt'

# Funcion que recibe un archivo como argumento, lo procesa y devuelve el cuerpo de un json que se envia a traves de una peticion POST a la API
def process_file(file_path):
    #Se inicializan dos objetos de tipo counter y una lista vacia
    client_ips = Counter()
    queried_hosts = Counter()
    dns_queries = []

    with open(file_path, 'r') as archivo:
        for linea in archivo:
            parts = linea.strip().split()

            #Extraer las partes que deben ir en el cuerpo del json, segun documentacion de la API
            try:
                query_datetime = datetime.strptime(f"{parts[0]} {parts[1]}", "%d-%b-%Y %H:%M:%S.%f")
            except ValueError:
                continue  

            #client_ip = parts[13].strip('()')
            client_ip = parts[6].split('#')[0].strip('()')
            host_queried = parts[7].strip('():')  
            query_type = parts[11]  
            nombre_cliente = parts[5].strip('@')

            client_ips[client_ip] += 1
            queried_hosts[host_queried] += 1

            # Se realiza un append a la lista vacia creada en un comienzo
            dns_queries.append({
                "timestamp": query_datetime.isoformat() , 
                "name": host_queried,
                "client_ip": client_ip,
                "client_name": nombre_cliente,  
                "type": query_type
            })

    return dns_queries, client_ips, queried_hosts


#Funcion que divide el cuerpo del json en bloques de 500
def data_blocks(data, chunk_size=500):
    return [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]
    

#Funcion que envia datos en bloques de 500 a la API
def send_data_blocks(dns_queries):
    
    chunks = data_blocks(dns_queries, 500)
    
    for chunk in chunks:
        payload = chunk
        
        response = send_data_api(payload)
        if response.status_code == 200:
            print(f"Se envio bloque con {len(chunk)} registros.")
        else:
            print(f"Error al enviar bloque")
            

#Funcion que muestra las estadisticas solicitadas
def show_statistics(total_queries, client_ips, queried_hosts):
    print(f"\nTotal records: {total_queries}")

    print("\nClient IPs Rank:")
    for ip, count in client_ips.most_common(5):
        print(f"client_ip: {ip}, Consultas: {count}, Porcentaje: {count / total_queries * 100:.2f}%")

    print("\nHost Rank:")
    for host, count in queried_hosts.most_common(5):
        print(f"name: {host}, Consultas: {count}, Porcentaje: {count / total_queries * 100:.2f}%")



# se invoca la funcion process_file y devuelve 3 objetos
dns_queries, client_ips, queried_hosts = process_file(file_path)

# Se invoca la funcion para enviar datos en bloques de 500 a la API
send_data_blocks(dns_queries)

total_queries = len(dns_queries)

# Se invoca la funcion show_statistics
show_statistics(total_queries, client_ips, queried_hosts)




Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 500 registros.
Se envio bloque con 

In [17]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\PC\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
